In [1]:
pip install lda2vec

Note: you may need to restart the kernel to use updated packages.


In [1]:
#FOR TEXT PREPROCESSING
import re
import spacy

import matplotlib.pyplot as plt
import seaborn as sns
import time

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import numpy for matrix operation
import numpy as np

# import pandas for dataframe operation
import pandas as pd

# Importing Gensim
import gensim
from gensim import corpora, models


from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer= WordNetLemmatizer()
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

# Enable logging
import logging
logging.basicConfig(level= logging.INFO)

In [2]:
# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
#Reading the data
df = pd.read_csv("C:\\Users\\arsen\\UWTacoma_Courses\\TCSS499\\Tawosi_Dataset\\ALOY-test.csv", error_bad_lines=False)
df

FileNotFoundError: [Errno 2] No such file or directory: 'ALOY-train.csv'

In [4]:
#Reading the data
df = pd.read_csv("C:\\Users\\arsen\\UWTacoma_Courses\\TCSS499\\Tawosi_Dataset\\ALOY-test.csv", error_bad_lines=False)
df

,issuekey,type,components,storypoint,title,description_text
0,ALOY-1504,Bug,Builtins,3,ERROR: Alloy is not defined,"""I have an Android project integrated with _ia..."
1,ALOY-1508,Bug,Tooling,3,iOS: Localised splash screen is not visible,"""When I add the splash screen in the """"app/i18..."
2,ALOY-1502,Bug,Models,8,Destroy Model in Alloy doesn't seem to be working,"""Hi, Trying to delete models using destroy() ..."
3,ALOY-1505,Improvement,XML,3,Add ability to pass cancel attribute to Option...,"""{quote}As a developer, I want to be able to ..."
4,ALOY-1510,Bug,XML,1,"Receiving ""<SearchView> is only available in A...","""h2. Problem We have a number of views with t..."
5,ALOY-1517,Bug,Styling,3,UI elements are not created properly with form...,"""When I use a conditional style I know the sty..."
6,ALOY-1512,Story,Tooling,0,Replace Uglify with Babylon,"""At the moment, Uglify has limited support of ..."
7,ALOY-1528,Bug,Widgets,2,getWidgetDirectories does not respect theme co...,"""We have multiple themes for our Alloy/Titaniu..."
8,ALOY-1519,Improvement,XML,5,Alloy: Support iOS 10 <RefreshControl> tag in ...,"""We introduced the {{Ti.UI.RefreshControl}} us..."
9,ALOY-1520,Bug,Tooling,3,Replace Ti.Ui.iPhone with Ti.Ui.iOS in Alloy,"""Since 5.4.0.GA we facing a lot of warnings me..."


In [5]:
df[['title','description_text']].groupby(['title','description_text']).agg('count')

,
title,description_text
ALLOY: Model Data binding problem with the first upper case in the model src,"""There is a problem with code generated for Simple Data Binding with the first upper case letter in model name. I have two models defined: app/models/book.js app/models/Task.js I expect that following XMLs will produce the same code: {code:xml} <Alloy> <Model src=""""book""""/> <Label>{book.name}</Label> </Alloy> {code} {code:xml} <Alloy> <Model src=""""Task""""/> <Label>{Task.name}</Label> </Alloy> {code} But in the first case it is correct: {code:javascript} Alloy.Models.instance('book'); $.__views.Book = Ti.UI.createLabel({ id: """"Book"""" }); $.__views.Book && $.addTopLevelView($.__views.Book); var __alloyId2 = function () { Alloy['Models']['book'].__transform = _.isFunction(Alloy['Models']['book'].transform) ? Alloy['Models']['book'].transform() : Alloy['Models']['book'].toJSON();$.Book.text = Alloy['Models']['book']['__transform']['name']; };Alloy['Models']['book'].on('fetch change destroy', __alloyId2);exports.destroy = function () { Alloy['Models']['book'] && Alloy['Models']['book'].off('fetch change destroy', __alloyId2); }; {code} and in the second case it is invalid: {code:javascript} $.__views.Task = Ti.UI.createLabel({ text: $model.__transform.Task.name, id: """"Task"""" }); $.__views.Task && $.addTopLevelView($.__views.Task); {code} and I get error $model.__transform is undefined"""
Add ability to pass cancel attribute to Option element for OptionDialog,"""{quote}As a developer, I want to be able to mark an option as being the option to use for cancelling an OptionDialog in Alloy XML, So that I can use OptionDialog in view without dependency on controller. {quote} Currently, I can do this: However, if an option is not displayed due to conditional code, then the index number will no longer be valid. Proposed new attribute: """
Add alloy.js plugin file to .gitignore,"""Every time the alloy version changes, the alloy.js plugin file inside the plugin folder changes too. This pollutes git and a default procedure is to remove the file from git alltogether. This is discussed in a recent kitchensink PR https://github.com/appcelerator/kitchensink-v2/pull/31#pullrequestreview-101563655 by [~hknoechel], [~eharris] and me. """
Add optimizingExeptions option to config.json,"""This PR make you can skip the file when doing an optimization manually. There are no reason to optimize js files which doesn't depends on alloy. Community PR : https://github.com/appcelerator/alloy/pull/818"""
Add support for Backbone 1.3.3 and Underscore 1.8.3,"""Currently Alloy uses backbone 0.9.2 as default, and there is an ability to upgrade to 1.1.2 through the config.json as per documentation: [https://wiki.appcelerator.org/display/guides2/Alloy+Backbone+Migration]. However, the current release of backbone is 1.3.3. There should be support for this version in Alloy too. And I assume it wouldn't hurt updating the default backbone version either, but I guess thats a breaking change and perhaps another ticket. """
Allow Android to use Ti.UI.RefreshControl as well,"""We added refresh-control for Android in TIMOB-1072, but we also need Android to use the Alloy <RefreshControl> tag."""
Alloy doesn't compile external app.js,"""If your Alloy project has an `app.js` in a subdirectory (example: `lib/app.js`), it is not compiled. So, if you have a pre-processor constant like `OS_IOS` or `OS_ANDROID` in this file, the result is a runtime error because no var defined `OS_IOS` is found. *This is a critical issue, because Alloy produces a broken app!* PR: https://github.com/appcelerator/alloy/pull/836"""
Alloy uses old require-paths with new require-behavior,"""We introduced an updated require-handling in Titanium 6 that matches the full NodeJS specs for handling all kind of files and data structures. Although we are save for older require-paths, we should update the Alloy-internal require-calls as well. Example log: It will only be shown in the DEBUG log-leve

In [6]:
# Remove issuekey, type and components from dataframe
# issuekey is irrelevant independent variable for our project
# in the first step of our project (in topic modelling part), type and components are not considered
def prepare_data(dataframe):
    dataframe = dataframe.drop('issuekey', axis=1)
    dataframe = dataframe.drop('type', axis=1)
    dataframe = dataframe.drop('components', axis=1)
# To capture the context of issue reports and their purpose, the title and description of the issue are combined, 
# dubbed the issue-context
    dataframe['issue_context'] = dataframe['title'] + dataframe['description_text']
# Remove title and description_text from dataframe
    dataframe = dataframe.drop('title', axis=1)
    dataframe = dataframe.drop('description_text', axis=1)
    # Since the issue-context is our main content, handling the missing values in' issue-context' which is null
    dataframe.fillna(method ='ffill', inplace = True)
    return dataframe

In [7]:
df=prepare_data(df)
# Take a peek of the dataframe
print(len(df))
print(df[:5])

50
   storypoint                                      issue_context
0           3  ERROR: Alloy is not defined"I have an Android ...
1           3  iOS: Localised splash screen is not visible"Wh...
2           8  Destroy Model in Alloy doesn't seem to be work...
3           3  Add ability to pass cancel attribute to Option...
4           1  Receiving "<SearchView> is only available in A...


In [8]:
def preprocessing(dataframe):
    # REMOVE URLs from the issue-context
    pattern = r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}[-a-zA-Z0-9()@:%_+.~#?&/=]*)' 
    dataframe['issue_context'] = dataframe['issue_context'].apply(lambda x: re.split(pattern, str(x))[0])
    # REMOVE all-non-alphanumeric characters from the issue-context
    #Function used internally to replace all non-alphanumeric characters.
    alpha_numeric= r'[^[:alnum:]]' 
    dataframe['issue_context'] = dataframe['issue_context'].str.replace(alpha_numeric, ' ')
    # Convert the text to lowercase
    dataframe['issue_context']=dataframe['issue_context'].str.lower()
    # Remove words less than 2
    dataframe['issue_context']=dataframe['issue_context'].str.replace(r'\b(\w{1,2})\b', '')
    # Remove punctuation
    dataframe['issue_context'] = dataframe['issue_context'].str.replace('[^\w\s]',' ')
    # REMOVE STOPWORDS
    # Import stopwords with nltk.
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    print(stop_words)
    dataframe['issue_context'] = dataframe['issue_context'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return dataframe

In [9]:
df=preprocessing(df)
df

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arsen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,storypoint,issue_context
0,3,error alloy defined android project integrated...
1,3,ios localised splash screen visible add splash...
2,8,destroy model alloy seem working trying delete...
3,3,add ability pass cancel attribute option eleme...
4,1,receiving searchview available android compili...
5,3,elements created properly formfactor property ...
6,0,replace uglify babylon moment uglify limited s...
7,2,getwidgetdirectories respect theme config json...
8,5,alloy support ios refreshcontrol tag scrollvie...
9,3,replace iphone ios alloy since facing lot warn...


In [10]:
def find_corpus(dataframe):
    corpus = []
    corpus =  list(dataframe['issue_context'])
    return corpus

In [11]:
corpus=find_corpus(df)
corpus

['error alloy defined android project integrated _iamyellow net_ push notification module erroneous scenario generated following steps app closed manually background notification notification bar app app opened tapping notification error alloy defined sourced build android assets alloy appeared please refer screenshot attachment app sticks default screen receiving error please consider high priority issue regards',
 'ios localised splash screen visible add splash screen app i18n path able see splash screen get black screen steps reproduce create alloy project place splash screen image app i18n folder run app simulator expected splash screen visible app launches actual black screen shown instead splash screen image reference',
 'destroy model alloy seem working trying delete models using destroy events seems working steps reproduce create simple alloy project create model testuser copy following code index run project destroy model alloy seem working get log destroy log environment than

In [12]:
# lemmatization
lemma = WordNetLemmatizer() 
# One function for all the steps:
def clean(doc):
    # normalize the text
    normalized = " ".join([lemma.lemmatize(word) for word in doc.split()])  
    return normalized

In [13]:
# clean data stored in a new list
clean_corpus = [clean(doc).split() for doc in corpus] 
clean_corpus

LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/wordnet[0m

  Searched in:
    - 'C:\\Users\\arsen/nltk_data'
    - 'C:\\Users\\arsen\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'C:\\Users\\arsen\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'C:\\Users\\arsen\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\arsen\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [14]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arsen\AppData\Roaming\nltk_data...


True

In [17]:
# clean data stored in a new list
clean_corpus = [clean(doc).split() for doc in corpus] 
clean_corpus

[['error',
  'alloy',
  'defined',
  'android',
  'project',
  'integrated',
  '_iamyellow',
  'net_',
  'push',
  'notification',
  'module',
  'erroneous',
  'scenario',
  'generated',
  'following',
  'step',
  'app',
  'closed',
  'manually',
  'background',
  'notification',
  'notification',
  'bar',
  'app',
  'app',
  'opened',
  'tapping',
  'notification',
  'error',
  'alloy',
  'defined',
  'sourced',
  'build',
  'android',
  'asset',
  'alloy',
  'appeared',
  'please',
  'refer',
  'screenshot',
  'attachment',
  'app',
  'stick',
  'default',
  'screen',
  'receiving',
  'error',
  'please',
  'consider',
  'high',
  'priority',
  'issue',
  'regard'],
 ['io',
  'localised',
  'splash',
  'screen',
  'visible',
  'add',
  'splash',
  'screen',
  'app',
  'i18n',
  'path',
  'able',
  'see',
  'splash',
  'screen',
  'get',
  'black',
  'screen',
  'step',
  'reproduce',
  'create',
  'alloy',
  'project',
  'place',
  'splash',
  'screen',
  'image',
  'app',
  'i18n',


In [16]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\arsen\AppData\Roaming\nltk_data...


True

In [18]:
# Creating the term dictionary of our courpus that is of all the words (Sepcific to Genism syntax perspective), 
# where every unique term is assigned an index. 
dictionary = corpora.Dictionary(clean_corpus)
print(dictionary)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:built Dictionary<658 unique tokens: ['_iamyellow', 'alloy', 'android', 'app', 'appeared']...> from 50 documents (total 1878 corpus positions)
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<658 unique tokens: ['_iamyellow', 'alloy', 'android', 'app', 'appeared']...> from 50 documents (total 1878 corpus positions)", 'datetime': '2022-12-18T18:50:57.436599', 'gensim': '4.2.0', 'python': '3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}


Dictionary<658 unique tokens: ['_iamyellow', 'alloy', 'android', 'app', 'appeared']...>


In [19]:
# The dictionary had 1238 unqiue words/tokens in the cleaned corpus.
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > len(dictionary):
        break

0 _iamyellow
1 alloy
2 android
3 app
4 appeared
5 asset
6 attachment
7 background
8 bar
9 build
10 closed
11 consider
12 default
13 defined
14 erroneous
15 error
16 following
17 generated
18 high
19 integrated
20 issue
21 manually
22 module
23 net_
24 notification
25 opened
26 please
27 priority
28 project
29 push
30 receiving
31 refer
32 regard
33 scenario
34 screen
35 screenshot
36 sourced
37 step
38 stick
39 tapping
40 able
41 actual
42 add
43 black
44 create
45 expected
46 folder
47 get
48 i18n
49 image
50 instead
51 io
52 launch
53 localised
54 path
55 place
56 reference
57 reproduce
58 run
59 see
60 shown
61 simulator
62 splash
63 visible
64 code
65 copy
66 delete
67 destroy
68 environment
69 event
70 index
71 log
72 model
73 seem
74 seems
75 simple
76 testuser
77 thanks
78 trying
79 using
80 working
81 ability
82 attribute
83 cancel
84 cancelling
85 conditional
86 controller
87 currently
88 dependency
89 developer
90 displayed
91 due
92 element
93 however
94 longer
95 mark
96 ne

In [20]:
# Converting list of documents (corpus) into Document Term Matrix using the dictionary 
bow_doc_term_matrix = [dictionary.doc2bow(i) for i in clean_corpus]
bow_doc_term_matrix

[[(0, 1),
  (1, 3),
  (2, 2),
  (3, 4),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 3),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 4),
  (25, 1),
  (26, 2),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1)],
 [(1, 1),
  (3, 4),
  (28, 1),
  (34, 8),
  (37, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 2),
  (49, 2),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 6),
  (63, 2)],
 [(1, 3),
  (16, 1),
  (28, 2),
  (37, 1),
  (44, 2),
  (47, 1),
  (57, 1),
  (58, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 4),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 2),
  (72, 4),
  (73, 2),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),

In [21]:
len(bow_doc_term_matrix)

50

In [22]:
bow_doc_term_matrix[49]

[(1, 2),
 (64, 1),
 (79, 2),
 (154, 1),
 (224, 2),
 (258, 2),
 (269, 1),
 (447, 2),
 (653, 1),
 (654, 2),
 (655, 1),
 (656, 1),
 (657, 1)]

In [1]:
from lda2vec import preprocess, Corpus
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

try:
    import seaborn
except:
    pass


In [6]:
pip install chainer

     ---------------------------------------- 1.0/1.0 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 527.0/527.0 kB 4.7 MB/s eta 0:00:00
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=967725 sha256=fb41a03cd07a9b321dbbe6b48ee275c222cd03fcdf93fa44f00efcb2f67823f8
  Stored in directory: c:\users\arsen\appdata\local\pip\cache\wheels\b5\ea\50\c75652ef47edc9ba9d5e0ba890fa2fbb12240e018b3474fced
Successfully built chainer
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install -U pip setuptools wheel

     ---------------------------------------- 2.1/2.1 MB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 9.8 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.1
    Uninstalling wheel-0.37.1:
      Successfully uninstalled wheel-0.37.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 65.5.0
    Uninstalling setuptools-65.5.0:
      Successfully uninstalled setuptools-65.5.0
  Attempting uninstall: pip
    Found existing installation: pip 22.3
    Uninstalling pip-22.3:
      Successfully uninstalled pip-22.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install -U spacy

     ---------------------------------------- 11.9/11.9 MB 9.8 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.2
    Uninstalling spacy-3.4.2:
      Successfully uninstalled spacy-3.4.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

  Using cached tensorflow-2.11.0-cp310-cp310-win_amd64.whl (1.9 kB)
Note: you may need to restart the kernel to use updated packages.


  Using cached tensorflow-2.11.0-cp310-cp310-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.11.0-cp310-cp310-win_amd64.whl (266.3 MB)


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\arsen\\AppData\\Local\\Programs\\Python\\Python310\\Lib\\site-packages\\tensorflow\\python\\keras\\mixed_precision\\loss_scale.py'
Consider using the `--user` option or check the permissions.



In [3]:
pip uninstall tensorflow

^C
Note: you may need to restart the kernel to use updated packages.


In [24]:
from datetime import datetime
import os
import sys

import numpy as np
import tensorflow as tf
from tensorboard.plugins import projector

from lda2vec import dirichlet_likelihood
from lda2vec import EmbedMixture
from lda2vec import NegativeSampling
from lda2vec import utils

In [7]:
DEFAULTS = {
		"n_document_topics": 15,
		"n_embedding": 100, # embedding size

		"batch_size": 500,
		"window": 5,
		"learning_rate": 1E-3,
		"dropout_ratio": 0.8, # keep_prob
		"word_dropout": 0.8, #1.

		"power": 0.75, # unigram sampler distortion
		"n_samples": 50, # num negative samples

		"temperature": 1., # embed mixture temp
		"lmbda": 200., # strength of Dirichlet prior
		"alpha": None, # alpha of Dirichlet process (defaults to 1/n_topics)
	}
RESTORE_KEY = "to_restore"

In [8]:
def __init__(self, n_documents=None, n_vocab=None, d_hyperparams={},
				 freqs=None, w_in=None, fixed_words=False, word2vec_only=False,
				 meta_graph=None, save_graph_def=True, log_dir="./log"):
		"""
		Initialize LDA2Vec model from parameters or saved `meta_graph`
		Args:
		    n_documents (int)
		    n_vocab (int)
		    d_hyperparams (dict): model hyperparameters
		    freqs (list or np.array): iterable of word frequencies for candidate sampler
		                              (None defaults to unigram sampler)
		    w_in (np.array): pre-trained word embeddings (n_vocab x n_embedding)
		    fixed_words (bool): train doc and topic weights only?
		    word2vec_only (bool): word2vec context and objective only?
		    meta_graph (str): path/to/saved/meta_graph (without `.meta`)
		    save_graph_def (bool): save graph_def to file?
		    log_dir (str): path/to/logging/outputs
		"""

		assert not (n_documents is None and n_vocab is None and meta_graph is None), (
				"Must initialize new model or pass saved meta_graph")
		assert not (fixed_words and w_in is None), (
				"If fixing words, must supply pre-trained word embeddings")
		assert not (fixed_words and word2vec_only), ("Nothing to train here...")

		self.__dict__.update(LDA2Vec.DEFAULTS, **d_hyperparams)
		tf.reset_default_graph()
		self.sesh = tf.Session()
		self.moving_avgs = tf.train.ExponentialMovingAverage(0.9)

		if not meta_graph: # new model
			self.datetime = datetime.now().strftime(r"%y%m%d_%H%M")

			# build graph
			self.mixture = EmbedMixture(
					n_documents, self.n_document_topics, self.n_embedding,
					temperature=self.temperature)

			# optionally, pass in pre-trained non/trainable word embeddings
			if w_in is not None:
				assert n_vocab == w_in.shape[0], "Word embeddings must match vocab size"
			W_in = (w_in if w_in is None else (tf.constant(w_in) if fixed_words
											   else tf.Variable(w_in)))
			self.sampler = NegativeSampling(
					self.n_embedding, n_vocab, self.n_samples, power=self.power,
					freqs=freqs, W_in=W_in)

			handles = self._buildGraph(word2vec_only=word2vec_only) + (
				self.mixture(), self.mixture.proportions(softmax=True),
				self.mixture.factors, self.sampler.W)

			for handle in handles:
				tf.add_to_collection(LDA2Vec.RESTORE_KEY, handle)
				self.sesh.run(tf.global_variables_initializer())

		else: # restore saved model
			datetime_prev, _ = os.path.basename(meta_graph).split("_lda2vec")
			datetime_now = datetime.now().strftime(r"%y%m%d_%H%M")
			self.datetime = "{}_{}".format(datetime_prev, datetime_now)

			# rebuild graph
			meta_graph = os.path.abspath(meta_graph)
			tf.train.import_meta_graph(meta_graph + ".meta").restore(
				self.sesh, meta_graph)
			handles = self.sesh.graph.get_collection(LDA2Vec.RESTORE_KEY)

		# unpack tensor ops to feed or fetch
		(self.pivot_idxs, self.doc_at_pivot, self.dropout, self.target_idxs,
		 self.fraction, self.loss_word2vec, self.loss_lda, self.loss,
		 self.global_step, self.train_op, self.switch_loss, self.doc_embeds,
		 self.doc_proportions, self.topics, self.word_embeds) = handles

		self.log_dir = "{}_{}".format(log_dir, self.datetime)
		if save_graph_def: # tensorboard
			self.logger = tf.summary.FileWriter(self.log_dir, self.sesh.graph)


In [9]:
@property
def step(self):
	"""Train step"""
	return self.sesh.run(self.global_step)

In [10]:
def _buildGraph(self, word2vec_only=False):

		global_step = tf.Variable(0, trainable=False)

		# pivot word
		pivot_idxs = tf.placeholder(tf.int32,
									shape=[None,], # None enables variable batch size
									name="pivot_idxs")
		pivot = tf.nn.embedding_lookup(self.sampler.W, pivot_idxs) # word embedding

		# doc
		doc_at_pivot = tf.placeholder(tf.int32, shape=[None,], name="doc_ids")
		doc = self.mixture(doc_at_pivot) # doc embedding

		# context is sum of doc (mixture projected onto topics) & pivot embedding
		dropout = self.mixture.dropout
		switch_loss = tf.Variable(0, trainable=False)

		# context = tf.nn.dropout(doc, dropout) + tf.nn.dropout(pivot, dropout)
		contexts = (tf.nn.dropout(pivot, dropout), tf.nn.dropout(doc, dropout))
		context = (tf.cond(global_step < switch_loss,
						  lambda: contexts[0],
						  lambda: tf.add(*contexts)) if not word2vec_only
				   else contexts[0])

		# targets
		target_idxs = tf.placeholder(tf.int64, shape=[None,], name="target_idxs")

		# NCE loss
		with tf.name_scope("nce_loss"):
			loss_word2vec = self.sampler(context, target_idxs)
			loss_word2vec = utils.print_(loss_word2vec, "loss_word2vec")

		# dirichlet loss (proportional to minibatch fraction)
		with tf.name_scope("lda_loss"):
			fraction = tf.Variable(1, trainable=False, dtype=tf.float32)
			#loss_lda = fraction * self.prior() # dirichlet log-likelihood
			loss_lda = self.lmbda * fraction * self.prior() # dirichlet log-likelihood
			loss_lda = utils.print_(loss_lda, "loss_lda")

		# optimize
		#loss = tf.identity(loss_word2vec + self.lmbda * loss_lda, "loss")
		# loss = tf.identity(loss_word2vec + loss_lda, "loss")
		loss = (tf.cond(global_step < switch_loss,
					   lambda: loss_word2vec,
					   lambda: loss_word2vec + loss_lda) if not word2vec_only
					   # lambda: loss_word2vec + self.lmbda * loss_lda)
				else tf.identity(loss_word2vec)) # avoid duplicating moving avg (ValueError)

		loss_avgs_op = self.moving_avgs.apply([loss_lda, loss_word2vec, loss])

		with tf.control_dependencies([loss_avgs_op]):
			train_op = tf.contrib.layers.optimize_loss(
					loss, global_step, self.learning_rate, "Adam", clip_gradients=5.)

		return (pivot_idxs, doc_at_pivot, dropout, target_idxs, fraction,
				loss_word2vec, loss_lda, loss, global_step, train_op, switch_loss)

In [11]:
def prior(self):
		# defaults to inialization with uniform prior (1/n_topics)
		return dirichlet_likelihood(self.mixture.W, alpha=self.alpha)

In [12]:
def _addSummaries(self, metadata="metadata.tsv",
					  metadata_docs="metadata.docs.tsv"):
		# summary nodes
		tf.summary.scalar("loss_lda", self.loss_lda)
		tf.summary.scalar("loss_nce", self.loss_word2vec)

		tf.summary.scalar("loss_lda_avg", self.moving_avgs.average(self.loss_lda))
		tf.summary.scalar("loss_nce_avg", self.moving_avgs.average(self.loss_word2vec))
		tf.summary.scalar("loss_avg", self.moving_avgs.average(self.loss))

		tf.summary.histogram("word_embeddings_hist", self.word_embeds)
		tf.summary.histogram("topic_embeddings_hist", self.topics)
		tf.summary.histogram("doc_embeddings_hist", self.doc_embeds)

		tf.summary.scalar("doc_mixture_sparsity",
						  tf.nn.zero_fraction(self.doc_proportions))

		# viz
		config = projector.ProjectorConfig()

		embedding = config.embeddings.add()
		embedding.tensor_name = self.word_embeds.name
		embedding.metadata_path = os.path.join(self.log_dir, metadata)

		topic_embedding = config.embeddings.add()
		topic_embedding.tensor_name = self.topics.name

		doc_embedding = config.embeddings.add()
		doc_embedding.tensor_name = self.doc_embeds.name
		doc_embedding.metadata_path = os.path.join(self.log_dir, metadata_docs)

		doc_props = config.embeddings.add()
		doc_props.tensor_name = self.doc_proportions.name
		doc_props.metadata_path = os.path.join(self.log_dir, metadata_docs)

		projector.visualize_embeddings(self.logger, config)

		return tf.summary.merge_all()

In [13]:
def make_feed_dict(self, doc_ids, word_indices, window=None,
					   update_only_docs=False):

		window = (self.window if window is None else window)
		pivot_idx = word_indices[window: -window]
		doc_at_pivot = doc_ids[window: -window]

		start, end = window, word_indices.shape[0] - window

		target_idxs = []

		for frame in range(-window, window + 1):

			# Skip predicting the current pivot
			if frame == 0:
				continue

			# Predict word given context and pivot word
			# The target starts before the pivot
			target_idx = word_indices[start + frame: end + frame]
			doc_at_target = doc_ids[start + frame: end + frame]
			doc_is_same = doc_at_target == doc_at_pivot

			rand = np.random.uniform(0, 1, doc_is_same.shape[0])
			mask = (rand < self.word_dropout)
			weight = np.logical_and(doc_is_same, mask).astype(np.int32)

			# If weight is 1.0 then targetidx
			# If weight is 0.0 then -1
			target_idx = target_idx * weight + -1 * (1 - weight)

			target_idxs.append(target_idx)

		pivot_idxs = np.tile(pivot_idx, window * 2)
		docs_at_pivot = np.tile(doc_at_pivot, window * 2)
		target_idxs = np.concatenate(target_idxs)

		# ignore training points due to OOV or dropout
		# TODO set OOV token globally
		LAST_OOV_TOKEN = 1
		# mask = np.logical_and((target_idxs > 0), (pivot_idxs > 0))
		mask = np.logical_and((target_idxs > LAST_OOV_TOKEN),
							  (pivot_idxs > LAST_OOV_TOKEN))
		# assert sum(mask) > 0, "At least one example must not be masked"

		feed_dict = {self.pivot_idxs: pivot_idxs[mask],
					 self.doc_at_pivot: docs_at_pivot[mask],
					 self.target_idxs: target_idxs[mask],
					 self.dropout: self.dropout_ratio}

		return feed_dict

In [14]:
def train(self, doc_ids, flattened, max_epochs=np.inf, verbose=False,
			  loss_switch_epochs=0, # num epochs until LDA loss switched on
			  save=False, save_every=1000, outdir="./out", summarize=True,
			  summarize_every=1000, metadata="metadata.tsv",
			  metadata_docs="metadata.docs.tsv"):

		if save:
			try:
				os.mkdir(outdir)
			except(FileExistsError):
				pass
			saver = tf.train.Saver(tf.global_variables())
			outdir = os.path.abspath(self.log_dir)

		if summarize:
			try:
				self.logger.flush()
			except(AttributeError): # not yet logging
				self.logger = tf.summary.FileWriter(self.log_dir, self.sesh.graph)
			merged = self._addSummaries(metadata, metadata_docs)

		j = 0
		epoch = 0

		fraction = self.batch_size / len(flattened) # == batch / n_corpus
		self.sesh.run(tf.assign(self.fraction, fraction))

		# turn on LDA loss after n iters of training
		iters_per_epoch = (int(len(flattened) / self.batch_size) +
						   np.ceil(len(flattened) % self.batch_size))
		n = iters_per_epoch * loss_switch_epochs
		self.sesh.run(tf.assign(self.switch_loss, n))

		now = datetime.now().isoformat()[11:]
		print("------- Training begin: {} -------\n".format(now))

		while epoch < max_epochs:
			try:

				# doc_ids, word_idxs
				for d, f in utils.chunks(self.batch_size, doc_ids, flattened):
					t0 = datetime.now().timestamp()

					feed_dict = self.make_feed_dict(d, f)

					# if len(feed_dict[self.pivot_idxs]) == 0:
					# 	print("Empty batch. Skipping...")
					# 	continue

					fetches = [self.loss_lda, self.loss_word2vec,
							   self.loss, self.train_op]
					loss_lda, loss_word2vec, loss, _ = self.sesh.run(
						fetches, feed_dict=feed_dict)

					j += 1

					if verbose and j % 1000 == 0:
						msg = ("J:{j:05d} E:{epoch:05d} L_nce:{l_word2vec:1.3e} "
							   "L_dirichlet:{l_lda:1.3e} R:{rate:1.3e}")

						t1 = datetime.now().timestamp()
						dt = t1 - t0
						rate = self.batch_size / dt
						logs = dict(l_word2vec=loss_word2vec, epoch=epoch, j=j,
									l_lda=loss_lda, rate=rate)

						print(msg.format(**logs))

					if save and j % save_every == 0:
						outfile = os.path.join(outdir,
											   "{}_lda2vec".format(self.datetime))
						saver.save(self.sesh, outfile, global_step=self.step)

					if summarize and j % summarize_every == 0:
						summary = self.sesh.run(merged, feed_dict=feed_dict)
						self.logger.add_summary(summary, global_step=self.step)

				epoch += 1

			except(KeyboardInterrupt):
				break

		print("epoch", epoch)
		print("max", max_epochs)
		now = datetime.now().isoformat()[11:]
		print("------- Training end: {} -------\n".format(now))

		if save:
			outfile = os.path.join(outdir, "{}_lda2vec".format(self.datetime))
			saver.save(self.sesh, outfile, global_step=self.step)

		try:
			self.logger.flush()
			self.logger.close()
		except(AttributeError): # not logging
			pass

		sys.exit(0)

In [18]:
def _buildGraph_similarity(self):
		"""Build nodes to compute the cosine similarity between examples
		(doc/word/topic idxs) and corresponding embeddings
		"""
		idxs_in = tf.placeholder(tf.int32,
							  shape=[None,], # None enables variable batch size
							  name="idxs") # doc or topic or word

		n = tf.placeholder_with_default(10, shape=None, name="n")

		normalized_embedding = dict()
		for name, embedding in zip(("word", "topic", "doc"),
								   (self.word_embeds, self.topics, self.doc_embeds)):
			norm = tf.sqrt(tf.reduce_sum(embedding**2, 1, keep_dims=True))
			normalized_embedding[name] = embedding / norm

		similarities = dict()
		for in_, vs in (("word", "word"),
						("word", "topic"),
						("topic", "word"),
						("doc", "doc")):
			embeddings_in = tf.nn.embedding_lookup(normalized_embedding[in_],
												   idxs_in)
			similarity = tf.matmul(embeddings_in, normalized_embedding[vs],
								   transpose_b=True)
			values, top_idxs = tf.nn.top_k(similarity, sorted=True, k=n)
			# top_sims = tf.gather_nd(similarity, top_idxs)
			# similarities[(in_, vs)] = [top_idxs, top_sims]
			similarities[(in_, vs)] = (top_idxs, similarity)

		return (idxs_in, n, similarities)

In [19]:
def compute_similarity(self, ids, in_, vs, n=10):
		"""Compute the cosine similarity between minibatch examples
		and all embeddings.
		Args: ids (1-D array of idxs)
		      in_ = "doc" or "word" or "topic" (corresponding to ids)
		      vs = "doc" or "word" or "topic" (corresponding to embedding to compare)
		"""
		while True:
			try:
				feed_dict = {self.idxs_in: ids, self.n: n}
				fetches = self.similarities[(in_, vs)]
				top_idxs, sims = self.sesh.run(fetches, feed_dict=feed_dict)
				top_sims = sims[ # select similarity to top matching idxs per id
					tuple([i]*top_idxs.shape[1] for i in range(top_idxs.shape[0])),
					 top_idxs]
				return (top_idxs, top_sims)

			except(AttributeError): # not yet initialized
				(self.idxs_in, self.n,
				 self.similarities) = self._buildGraph_similarity()

In [6]:
import os
import sys

import numpy as np

In [7]:
def embeddings_from_meta_graph(meta_graph, outdir=None, tf_projector=False):
	"""Load trained model and save embeddings"""
	from model import lda2vec

	outdir = ("results_{}".format(os.path.basename(meta_graph))
			  if outdir is None else outdir)

	m = lda2vec(47, 47, meta_graph=meta_graph)

	topics = m.sesh.run(m.topics)
	doc_embeds = m.sesh.run(m.doc_embeds)
	doc_proportions = m.sesh.run(m.doc_proportions)
	word_embeds = m.sesh.run(m.word_embeds)

	for f, arr in (("topics", topics),
				   ("doc_embeds", doc_embeds),
				   ("doc_proportions", doc_proportions),
				   ("word_embeds", word_embeds)):

		np.save(os.path.join(outdir, f), arr)

In [8]:
def tf_projector_from_embeddings(f_embed, metadata="metadata.tsv", outdir=None):
	"""Load np array of trained embeddings and save for visualization with TensorBoard Projector"""
	import tensorflow as tf
	from tensorboard.plugins import projector

	outdir = (os.path.join(os.path.dirname(os.path.abspath(f_embed)), "out")
			  if outdir is None else outdir)

	# tf graph for embeddings
	embeds = tf.Variable([47], dtype=tf.float32, name="embeddings")
	embed_vals = tf.placeholder(tf.float32, [None, None])
	assign_embeds = tf.assign(embeds, embed_vals, validate_shape=False)

	sesh = tf.Session()
	sesh.run(tf.global_variables_initializer())
	logger = tf.summary.FileWriter(outdir, sesh.graph)

	# tf projector
	config = projector.ProjectorConfig()
	embedding = config.embeddings.add()
	embedding.tensor_name = embeds.name
	embedding.metadata_path = os.path.join(outdir, metadata)
	projector.visualize_embeddings(logger, config)

	# assign embeddings and save
	feed_dict = {embed_vals: np.load(f_embed)}
	sesh.run(assign_embeds, feed_dict=feed_dict)

	saver = tf.train.Saver(tf.global_variables())
	outfile, _ = os.path.splitext(os.path.basename(f_embed))
	saver.save(sesh, os.path.join(outdir, outfile))

	logger.flush()
	logger.close()

In [10]:
if __name__ == "__main__":
	infile = sys.argv[1:]

	# assert os.path.exists(infile + "meta"), "Where's your infile ?"
	# save_embeddings(meta_graph=infile)

	assert os.path.exists(infile), "Where's your infile ?"
	tf_projector_from_embeddings(f_embed=infile)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not list